In [23]:
from bs4 import BeautifulSoup as bs
import requests 
import pymongo
import pandas as pd
import re
import os
import csv
import time
import json
import ast
import pprint
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [24]:
def create_geojson(date):
    df_countries_by_day = df_merged[(df_merged.date==date)]
    df_data = df_countries_by_day.groupby(['country','country_2']).agg({'conf_count':'sum', 'cured_count':'sum', 'dead_count':'sum'}).reset_index()
    df_data = df_data.set_index('country')
                                              
    json_df = pd.read_json('static/world_geojsons/world.json', encoding='UTF-8')
    json_df_feat = pd.DataFrame(json_df.features)

    geo_dict = {}
    geo_string = ""

    for index, row in json_df_feat.iterrows():
#         print(row['features']['properties']['ADMIN'])
        try:
            str_feat_1 = ("{\"type\": \"Feature\",")

            prop_name = row['features']['properties']['ADMIN']
            prop_american_name = df_data.loc[prop_name,"country_2"]
            prop_confirmedCount = df_data.loc[prop_name,"conf_count"]
            prop_suspectedCount = 0
            prop_curedCount = df_data.loc[prop_name,"cured_count"]
            prop_deadCount = df_data.loc[prop_name,"dead_count"]    
            prop_date = date

            str_prop_double_quotes = str(row['features']['properties'])
            str_prop_double_quotes = str_prop_double_quotes.replace("\'","\"")

            str_prop_1 = ("\"properties\" : " + str_prop_double_quotes + "\",")[:-3]
            str_prop_2 = (",\"american_name\" : \"" + prop_american_name + "\",")
            str_prop_3 = ("\"confirmedCount\" : \"" + str(prop_confirmedCount) + "\",")
            str_prop_4 = ("\"suspectedCount\" : \"" + str(prop_suspectedCount) + "\",")
            str_prop_5 = ("\"curedCount\" : \"" + str(prop_curedCount) + "\",")
            str_prop_6 = ("\"deadCount\" : \"" + str(prop_deadCount) + "\",")
            str_prop_7 = ("\"date\" : \"" + prop_date + "\"},")

            str_prop_all  = str_prop_1 + str_prop_2 + str_prop_3 + str_prop_4 + str_prop_5 + str_prop_6 + str_prop_7
#             print(str_prop_all)
            str_geom_1 = ("\"geometry\":" + str(row['features']['geometry']) + "},")
            str_geom_1 = str_geom_1.replace("\'","\"")
#             print(str_geom_1)

            str_for_each_province = (str_feat_1)+(str_prop_all)+(str_geom_1)
#             print(str_for_each_province)
            geo_string = geo_string + (str_for_each_province)
        except:
            geo_string = geo_string
    
    pre_fix = "{\"type\": \"FeatureCollection\", \"features\": ["
    post_fix = "]}"

    total_string = pre_fix + geo_string[:-1] + post_fix

#     print(total_string)
    output_path = os.path.join("static/world_geojsons", date + ".json")
    with open(output_path, "w", encoding='UTF-8') as text_file:
        text_file.write(total_string)
        text_file.close()

In [25]:
# Get the data from 3 different data sources https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
df_confirmed = pd.read_csv(confirmed_url)
cured_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
df_cured = pd.read_csv(cured_url)
death_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
df_death = pd.read_csv(death_url)

df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,989,989,989,989,990,990,990,990,990,990
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,399,400,400,410,410,411,413,414,414,418
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,293,294,294,296,296,296,296,296,296,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,91,91,91


In [26]:
# Create unique rows by moving date column to row level for all Confirmed cases
df_confirmed_long = df_confirmed.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_confirmed_long.head()
output_path = os.path.join("static/world_data", "df_world_confirmed_original.csv")
df_confirmed.to_csv(output_path)

# Create unique rows by moving date column to row level for all Cured cases
df_cured_long = df_cured.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_cured_long.head()
output_path = os.path.join("static/world_data", "df_world_cured_original.csv")
df_cured.to_csv(output_path)

# Create unique rows by moving date column to row level for all Death cases
df_death_long = df_death.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_death_long.head()
output_path = os.path.join("static/world_data", "df_world_death_original.csv")
df_death.to_csv(output_path)

df_confirmed_long.head()
df_confirmed_long["new_date"] = pd.to_datetime(df_confirmed_long['Date'])

df_confirmed_long.head()

,Province/State,Country/Region,Lat,Long,Date,Value,new_date
0,Anhui,Mainland China,31.8257,117.2264,1/22/20,1,2020-01-22
1,Beijing,Mainland China,40.1824,116.4142,1/22/20,14,2020-01-22
2,Chongqing,Mainland China,30.0572,107.8740,1/22/20,6,2020-01-22
3,Fujian,Mainland China,26.0789,117.9874,1/22/20,1,2020-01-22
4,Gansu,Mainland China,36.0611,103.8343,1/22/20,0,2020-01-22


In [27]:
# If Province/State isn't filled, use the country value instead

df_confirmed_long["Province/State"].fillna(df_confirmed_long["Country/Region"], inplace=True)
df_confirmed_long = df_confirmed_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long", "Date":"date", "Value": "conf_count"})

df_cured_long["Province/State"].fillna(df_cured_long["Country/Region"], inplace=True)
df_cured_long = df_cured_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long","Date":"date", "Value": "cured_count"})

df_death_long["Province/State"].fillna(df_death_long["Country/Region"], inplace=True)
df_death_long = df_death_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long","Date":"date", "Value": "dead_count"})

In [28]:
# Make sure country names in both files match
df_confirmed_long.loc[(df_confirmed_long.country == 'Mainland China'),'country']='China'
df_confirmed_long.loc[(df_confirmed_long.country == 'US'),'country']='United States of America'
df_confirmed_long = df_confirmed_long.set_index(['date', 'american_name'])

df_cured_long.loc[(df_cured_long.country == 'Mainland China'),'country']='China'
df_cured_long.loc[(df_cured_long.country == 'US'),'country']='United States of America'
df_cured_long = df_cured_long.set_index(['date', 'american_name'])

df_death_long.loc[(df_death_long.country == 'Mainland China'),'country']='China'
df_death_long.loc[(df_death_long.country == 'US'),'country']='United States of America'
df_death_long = df_death_long.set_index(['date', 'american_name'])

In [29]:
# last_date = df_merged['date'].max()
# print(last_date)

# #### Keep for when having to rebuild all geojson. Go back to 2020-01-22
# last_date = last_date + timedelta(days=-2)
# print(last_date_new)

query_str = open('static/sql/world_max_date.sql')
query_text = ""
for text in query_str:
    query_text = query_text + text

connection_string = "postgres:postgres@localhost:5432/corona_db"
engine = create_engine(f'postgresql://{connection_string}')
connection = engine.connect()
# print(query_text) 

rs = connection.execute(query_text)
for i in rs:
    last_date = (i[0])
print(last_date)
connection.close()

2020-03-04


In [30]:
# Merge all 3 datasets
df_merged = pd.merge(df_confirmed_long, df_cured_long, how='left', on=['date', 'american_name', 'country','lat','long'])

df_merged = pd.merge(df_merged, df_death_long,how='left', on=['date', 'american_name', 'country','lat','long'])
df_merged = df_merged.reset_index()
df_merged = df_merged.drop(['date'], axis=1)
df_merged['country_2'] = df_merged['country']

# Remove this part and just df_merged to reload all available data
df_merged = df_merged.rename(columns={"new_date":"date"})

df_merged_new = df_merged[df_merged['date'] > last_date]
df_merged_new

,american_name,country,lat,long,conf_count,date,cured_count,dead_count,country_2


In [31]:
# Change df_world_new.csv to df_world_all.csv to reload all available data ALSO change df_merged_new to df_merged
output_path = os.path.join("static/world_data", "df_world_new.csv")
df_merged_new.to_csv(output_path)

df_merged_new = df_merged_new[df_merged['conf_count']!=0]

C:\Users\dutch\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [32]:
df_merged_new

,american_name,country,lat,long,conf_count,date,cured_count,dead_count,country_2


In [37]:
# Call create_geojson to create the geojson scripts
current_date = datetime.now()
print("Current Date ",current_date)
yesterday = current_date+ timedelta(days=-1)
print("Yesterday: ",yesterday)

start_date = datetime.strptime(last_date, "%Y-%m-%d")
# start_date = last_date
print("Start Date", start_date)

while yesterday > start_date:
    pass_date = start_date.strftime("%Y-%m-%d")
    print(pass_date)
    create_geojson(pass_date)
    start_date = start_date + timedelta(days=1)

Current Date  2020-03-05 12:53:47.033453
Yesterday:  2020-03-04 12:53:47.033453
Start Date 2020-03-04 00:00:00
2020-03-04


In [34]:
# Only save the new records to the database

def save_world_to_database(df_world):
    connection_string = "postgres:postgres@localhost:5432/corona_db"
    engine = create_engine(f'postgresql://{connection_string}')
#     connection = engine.connect()
#     connection.execute( '''TRUNCATE TABLE daily_stats_world''' )
#     connection.close()
    df_world.to_sql(name='daily_stats_world', con=engine, if_exists='append', index=False)

In [35]:
# Start here to process the world data (only call save_world_to_database)
url_world = "static/world_data/df_world_new.csv"
df_world = pd.read_csv(url_world)
df_world = df_world.drop(['country_2'], axis=1)
df_world.head()
save_world_to_database(df_world)

In [36]:
# TEST QUERY
# query_str = open('static/sql/test_world.sql')
query_str = open('static/sql/world_top_10.sql')
query_text = ""
for text in query_str:
    query_text = query_text + text

connection_string = "postgres:postgres@localhost:5432/corona_db"
engine = create_engine(f'postgresql://{connection_string}')    
print(query_text)
df_query = pd.read_sql_query(query_text, con=engine)

df_query

select  
	country, 
	sum(conf_count) total,
	sum(conf_count)-sum(cured_count)-sum(dead_count) sick, 
	sum(cured_count) cured, 
	sum(dead_count) dead,
	max(date)
    from daily_stats_world 
    where date = (select max(date ) from daily_stats_world) 
    group by country 
    order by total desc 
	limit 10


,country,total,sick,cured,dead,max
0,China,80271,27335,49955,2981,2020-03-04
1,South Korea,5621,5545,41,35,2020-03-04
2,Italy,3089,2706,276,107,2020-03-04
3,Iran,2922,2278,552,92,2020-03-04
4,Others,706,690,10,6,2020-03-04
5,Japan,331,282,43,6,2020-03-04
6,France,285,269,12,4,2020-03-04
7,Germany,262,246,16,0,2020-03-04
8,Spain,222,218,2,2,2020-03-04
9,United States of America,153,134,8,11,2020-03-04
